In [1]:
from jettool import tools
import pandas
import numpy

In [2]:
import requests
import json

In [29]:
list_names = 'https://api.tej.com.tw/info/category/list'

response = requests.get(list_names)
data = json.loads(response.text)['result']
list_data = { data[k].get('categoryName'):data[k] for k in data}
list_data.keys()

dict_keys(['試用資料庫', '公司財務面資料', '公司交易面資料', '債券資料庫', '總體經濟資料', '公司營運面資料', '衍生性金融商品資料庫', '信用風險分析', '基金資料庫'])

In [31]:
list_data.get('試用資料庫')

{'categoryId': 9,
 'categoryName': '試用資料庫',
 'icon': 'G09.png',
 'jumpUrl': 'datatables.html?db=TRAIL&t=試用資料庫',
 'ordinal': 9,
 'subs': None}

In [21]:
#所有的國別
db_names = 'https://api.tej.com.tw/info/database/list'
response = requests.get(db_names)
data = json.loads(response.text)['result']

In [24]:
table_names = 'https://api.tej.com.tw/info/tables/list'
response = requests.get(table_names)
data = json.loads(response.text)['result']
data.get('TWN')

[{'dbCode': 'TWN',
  'description': '【資料範圍】2000年開始<br />【資料來源】TEJ衍生<br />【資料頻率】日資料<br />【資料期間】2013年至今<br />',
  'enabled': True,
  'id': 'TWN/AAPRCDA',
  'name': '上市(櫃)調整股價(日)-均價',
  'tableCode': 'AAPRCDA'},
 {'dbCode': 'TWN',
  'description': '【資料範圍】以上市的證券及指數為收錄對象(TSE及OTC)<br />【資料頻率】月資料<br />',
  'enabled': True,
  'id': 'TWN/AAPRCM1',
  'name': '上市(櫃)調整股價(月)-除權息調整',
  'tableCode': 'AAPRCM1'},
 {'dbCode': 'TWN',
  'description': '【資料範圍】以上市的證券及指數為收錄對象(TSE及OTC)<br />【資料頻率】週資料<br />',
  'enabled': True,
  'id': 'TWN/AAPRCW1',
  'name': '上市(櫃)調整股價(週)-除權息調整',
  'tableCode': 'AAPRCW1'},
 {'dbCode': 'TWN',
  'description': '【資料範圍】以上市的證券及指數為收錄對象(TSE及OTC)<br />【資料期間】2000年至今<br />【資料頻率】年資料<br />',
  'enabled': True,
  'id': 'TWN/AAPRCY1',
  'name': '上市(櫃)調整股價(年)-除權息調整',
  'tableCode': 'AAPRCY1'},
 {'dbCode': 'TWN',
  'description': '【資料範圍】全國不動產租賃案件實際資訊<br />【資料期間】2008年至今(其中2008~201207為主動申報之歷史交易，約28000筆)<br />【資料頻率】2013/6/30以前，每月一次，於每月16日發佈；2013/7/1-2017/11/30改為每月2次，於每月1日、16日發佈；2017/12/1再為3次，於每

In [ ]:
tejtool = tools.financial_tool()

In [ ]:
tejtool.set_apikey("GDEy0mWAGqnI3EemCREGREZMcEVbnF")


### 一次查詢全部整合完成
#### 一個名稱、可到不同資料表查詢
#### 需正確定義名稱的精確差異

In [ ]:
#left_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','個別風險','長期反轉(T-60~T-13)','董監持股%','大股東持股(TSE)%','經理人持股%','單月營收(千元)']
left_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','單月營收(千元)']
data = tejtool.get_data(base_date='2019-12-31',window='2m',column_names=left_name)
#tejtool.save_data()
#data[data['coid']=='2330']

In [ ]:
tejtool.tempdata=self.tejapi.get("TWN/APRCD",coid=query_coid,mdate={'gte':mdate_down,'lte':mdate_up},opts={'sort':'mdate.desc','columns':query_columns}, paginate=True).rename(index=str, columns={'mdate':'zdate'})

In [ ]:
def calculate():
    tejtool.data['EP'] = tejtool.data['常續性稅後淨利']/tejtool.data['收盤價(元)']
    tejtool.data['常續性稅後淨利檢查'] = tejtool.check_above('EP',0,window='1d')
def evaluate():
    tejtool.data['購入'] = tejtool.check_condition(conditions=['常續性稅後淨利檢查'])
    tejtool.data['unit'] = tejtool.equal_pv()

In [ ]:
tejtool.back_test(back_interval='2019-10-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

In [ ]:
tejtool.tejapi.get(dataset_name,coid=query_coid,mdate={'gte':mdate_down,'lte':mdate_up},opts={'sort':'mdate.desc','columns':query_columns}, paginate=True).rename(index=str, columns={'mdate':'zdate'})